## LangChain Expression Language(LCEL)

https://python.langchain.com/v0.1/docs/expression_language/why/

### 기본 구조: 프롬프트 + 모델 + 출력 파서


In [18]:
# 필요한 패키지 설치
!pip install -Uq python-dotenv langchain_teddynote langchain_openai langchain langchain-community faiss-cpu

In [42]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv("../content/.env", override=True)

True

In [43]:
#API KEY 저장을 위한 os 라이브러리 호출
import os

os.environ['LANGCHAIN_PROJECT'] = 'LCEL'
print(f"[LANGCHAIN_PROJECT]\n{os.environ['LANGCHAIN_PROJECT']}")

[LANGCHAIN_PROJECT]
LCEL


In [44]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LCEL")

LangChain/LangSmith API Key가 설정되지 않았습니다. 참고: https://wikidocs.net/250954


### 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿
  - `template`: 템플릿 문자열. 문자열 내에서 중괄호 `{}`는 변수를 나타냄
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의함

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트

In [46]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [47]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [48]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [49]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [55]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o-mini",
    max_tokens=2048,
    temperature=0.1,
)

### [실습] 셰프 프롬프트 템플릿 만들기

1. 변수 `food`에 음식 식재료를 입력받기 
2. `food` 재료로 만들 수 있는 음식 메뉴를 추천해주는 프롬프트 템플릿 객체 `chef_prompt` 생성하기
3. `food` 변수에 재료(값)를 넣어서 프롬프트를 완성해보기

In [59]:
# 실습 코드 작성
# template 정의
template = "{food}로 만들 수 있는 음식 메뉴를 추천해주세요"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
chef_prompt = PromptTemplate.from_template(template)
chef_prompt

prompt = chef_prompt.format(food="토마토")

model.invoke(prompt)

AIMessage(content='토마토는 다양한 요리에 활용할 수 있는 재료입니다. 아래는 토마토로 만들 수 있는 몇 가지 음식 메뉴를 추천드립니다:\n\n1. **토마토 파스타**: 신선한 토마토 소스를 만들어 파스타와 함께 볶아내면 간단하면서도 맛있는 요리가 됩니다.\n\n2. **토마토 샐러드**: 신선한 토마토, 모짜렐라 치즈, 바질을 곁들여 올리브 오일과 발사믹 식초로 드레싱한 샐러드입니다.\n\n3. **토마토 수프**: 토마토를 갈아서 만든 수프로, 크림을 추가하면 더욱 부드럽고 풍미가 깊어집니다.\n\n4. **토마토 브루스케타**: 다진 토마토, 마늘, 바질을 섞어 바삭한 빵 위에 올려서 간단한 전채 요리로 즐길 수 있습니다.\n\n5. **토마토 리조또**: 토마토와 쌀을 함께 조리하여 만든 리조또로, 풍부한 맛을 느낄 수 있습니다.\n\n6. **토마토 오믈렛**: 계란과 함께 토마토를 넣어 만든 오믈렛으로, 아침 식사로 좋습니다.\n\n7. **토마토 피자**: 토마토 소스를 바른 피자 도우 위에 다양한 토핑을 올려 구워내면 맛있는 피자가 완성됩니다.\n\n8. **토마토 카레**: 토마토를 베이스로 한 카레로, 다양한 채소와 함께 조리하면 건강한 한 끼가 됩니다.\n\n이 외에도 토마토는 다양한 요리에 활용할 수 있으니, 창의력을 발휘해 보세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 396, 'prompt_tokens': 19, 'total_tokens': 415, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_t

### Chain 생성

#### LCEL(LangChain Expression Language)


```
chain = prompt | model | output_parser
```

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달


In [15]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatOpenAI()

chain = prompt | model
chain

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='{topic} 에 대해 쉽게 설명해주세요.')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001D1FE26F250>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001D1FE26F610>, root_client=<openai.OpenAI object at 0x000001D1FE26EFD0>, root_async_client=<openai.AsyncOpenAI object at 0x000001D1FE26F390>, model_kwargs={}, openai_api_key=SecretStr('**********'))

### [실습] 셰프 챗봇을 위한 openAI 모델 생성

1. 변수 `chef_model` 선언하기
2. 사용할 모델은 `gpt-4o-mini` 지정
3. 창의성을 조절하는 변수는 `0.1` 로 지정
4. 최대 활용할 토큰은 `4096`

In [ ]:
# 실습 코드 작성

### [실습] 셰프 챗봇을 위한 chef_chain 생성

- 프롬프트 템플릿 변수 `chef_prompt` 와 llm 모델을 담은 `chef_model` 모델을 체인(`|`)으로 연결하여 `chef_chain` 변수 생성

In [ ]:
# 실습 코드 작성

### invoke() 호출

- python 딕셔너리 형태(키: 값)로 입력값을 전달
- invoke() 함수 호출 시, 입력값을 전달

In [11]:
# input 딕셔너리에 주제 설정
input = {"topic": "인공지능의 학습 방법"}

In [ ]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달
# 이를 통해 AI 모델이 생성한 메시지를 반환
chain.invoke(input)

In [ ]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

### 출력파서(Output Parser)


In [14]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [ ]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser
chain

In [ ]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능의 학습 원리"}
chain.invoke(input)

In [ ]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

### [실습] 최종! 셰프 챗봇을 실행해보기

- 마지막 StrOutputParser()까지 추가하여 `chef_chain` 완성하기
- 변수 `food_input`에 딕셔너리 형태로 재료 넣어 `chef_chain` 실행해보기
- Ex) 두부, 김치, 당근 ...

In [1]:
# 실습

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 가능
- `model_name` 역시 변경하여 테스트가 가능

In [18]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화
model = ChatOpenAI(model_name="gpt-4o-mini")

# 문자열 출력 파서를 초기화
output_parser = StrOutputParser()

In [19]:
# 체인을 구성
chain = prompt | model | output_parser

In [ ]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

In [ ]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

In [ ]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

### [실습] 템플릿을 변경하여 나만의 여행 가이드 챗봇 만들기

- 위의 프롬프트를 아래 주제에 맞게 **변경** 해보기
1. 페르소나: 10년차 여행 가이드
2. 3일간 가성비 여행 계획을 세워주는 챗봇 생성
3. `{question}` 에는 여행갈 나라와 도시를 사용자에게 입력받음
4. `answer` 변수를 출력하여 챗봇의 답변 결과 확인
5. `Langsmith` 에 접속하여 실행 내용 확인

In [ ]:
tour_guide_template = """

"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성
prompt = PromptTemplate.from_template( )

# ChatOpenAI 챗모델을 초기화
model = ChatOpenAI(model_name="gpt-4o-mini")

# 문자열 출력 파서를 초기화
output_parser = StrOutputParser()

# 체인을 구성
chain = ----- | ---- | ----

In [ ]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 대한민국 서울을 여행하고 싶습니다."})
# 스트리밍 출력
stream_response(answer)

In [ ]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 이탈리아 로마를 여행하고 싶습니다."})
# 스트리밍 출력
stream_response(answer)